<a href="https://colab.research.google.com/github/canyue1111/TestProject/blob/master/faceCrop_emotion_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install anvil-uplink

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-cp37-none-any.whl size=45216 sha256=00f5db638436b45f9e50f8d7a5331fe4a91fa35352911484f3d954e599071fe4
  Stored in directory: /root/.cache/pip/wheels/a2/6e/4e/8b0ae12fb9b8a05715256952cf7609a8ab86285fab99b88c68
Successfully built ws4py


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      //await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
import dlib
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
from datetime import datetime
import pandas as pd
import time
from scipy.stats import linregress
import anvil.server

df = pd.DataFrame(columns =['Customers ID', 'Time', 'Emotion','FileName'] )
da = pd.DataFrame(columns =['Customers ID', 'Time', 'Emotion','FileName'] )
 #(0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)
 
emotion_class = ['Angry','Disgust','Fear', 'Happy', 'Sad','Surprise','Neutral']

model= load_model('ResNet-50.h5')

detector = dlib.get_frontal_face_detector()
CustomersID = 0
CustomerStatus = 0 # 0 means no facedetected, 1 means yes

In [ ]:
#Function to load the cropped image
def load_image(filename):
    img = load_img(filename, grayscale=False, target_size=(197, 197))
    #convert to array
    img = img_to_array(img)
    #reshape into a single sample with  channe 3
    img = img.reshape(1, 197, 197, 3)
    #prepare pixel data
    img = img.astype('float32')
    img = img / 255.0
    return img


In [ ]:
#crop the screenshot
def detect_faces(image):
    # Run detector and get bounding boxes of the faces on image.
    detected_faces = detector(image, 1)
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]

    return face_frames

In [ ]:
def trans_da(n):
  #(0=Angry, 1=Disgust, 2=Fear, 5=Happy, 2=Sad, 3=Surprise, 4=Neutral)
  if n == 'Angry':
    return 1
  if n == 'Disgust':
    return 1
  if n == 'Fear':
    return 2
  if n == 'Happy':
    return 5
  if n == 'Sad':
    return 2
  if n == 'Surprise':
    return 4
  if n == 'Neutral':
    return 3
  else: return n



In [ ]:
def check_Face_Exist(img_path):
  max_score = 0.99
  # Detect faces http://dlib.net/face_detector.py.html
  img = dlib.load_rgb_image(img_path)
  dets, scores, idx = detector.run(img, 1,-1)
  for i, d in enumerate(dets):
      #print("Detection {}, score: {}, face_type:{}".format(
      #    d, scores[i], idx[i]))
      if scores[i]> max_score:
        max_score = scores[i]
  if max_score > 1.0:
    return True
  else:
    return False


In [ ]:
anvil.server.connect("E6U72W7FISS5UWOI3GLS4YOX-N2IDQ4SYIEBLIFFM")

@anvil.server.callable
def trans_int_da():
  return df.to_dict('records')

@anvil.server.callable
def Customer_Performance(_id):
  try:
    id = int(_id)
    if id in da["Customers ID"].values:
      temp = da[da["Customers ID"] == id]
      slope, intercept, r_value, p_value, std_err = linregress(temp.index, temp['Emotion'].astype(float))
      title_temp = "Customer ID "+ str(id) + " Performance is "+ str(slope)
      return  title_temp
    else:  
      return "That "+ _id +" Customers ID NOT EXIST"
  except:
    return "Not Valid"

@anvil.server.callable
def lastThreeAngry():
  #last 3 emotion average is lower than 2.5 and all of them has negative emotion
  return (len(da.tail(3)[da['Emotion'] < 4]) == 3) & (da.tail(3)['Emotion'].mean()<2.5)

In [ ]:
update_ = True

img_path = 'photo.jpg'

while update_:
  time.sleep(2)

  filename = take_photo()

  if check_Face_Exist(img_path):
    print("Face Detected")
    CustomerStatus = 1
    image = io.imread(img_path)
    detected_faces = detect_faces(image)

  # Crop faces and plot
    for n, face_rect in enumerate(detected_faces):
      face = Image.fromarray(image).crop(face_rect)

      now = datetime.now()
      today = now.strftime("%H:%M:%S")
      #file name setting
      name = str(today) + '.png'
      #save file
      plt.savefig(name)

      # emotion_class[np.argmax(model.predict(load_image("file/" + name)))]
      addition=pd.DataFrame( {'Customers ID': [CustomersID],
        'Time': [today],'Emotion':  [emotion_class[np.argmax(model.predict(load_image(name)))]],'FileName':[name]})
      df = df.append(addition, ignore_index=True)
      addition['Emotion'] = addition['Emotion'].apply(trans_da)
      da = da.append(addition, ignore_index=True)
    plt.subplot(1, len(detected_faces), n+1)
    plt.axis('off')
    plt.imshow(face)
  
  else:
    print("NofaceDetected")
    if CustomerStatus == 1:
      CustomerStatus = 0
      CustomersID = CustomersID + 1



KeyboardInterrupt: ignored

In [ ]:
da

,Customers ID,Time,Emotion,FileName
0,0,14:33:04,4,14:33:04.png
1,0,14:33:05,4,14:33:05.png
2,0,14:33:12,4,14:33:12.png
3,0,14:33:13,4,14:33:13.png
4,0,14:33:20,4,14:33:20.png
...,...,...,...,...
163,12,15:06:40,4,15:06:40.png
164,12,15:06:44,4,15:06:44.png
165,12,15:06:44,4,15:06:44.png
166,12,15:06:49,4,15:06:49.png


In [ ]:
(len(da.tail(3)[da['Emotion'] < 4]) == 3) & (da.tail(3)['Emotion'].mean()<2.5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


False

#EDA


In [ ]:
import seaborn as sns

def Customer_Performance(_id,_dataframe):
  temp = _dataframe[_dataframe["Customers ID"] == _id]
  slope, intercept, r_value, p_value, std_err = stats.linregress(_dataframe.index,_dataframe['Emotion'])
  title_temp = "Customer ID "+ str(_id) + " Performance is "+ str(slope)
  sns.regplot(x=_dataframe.index, y="Emotion", data=_dataframe, fit_reg=True).set_title(title_temp)
  display(temp,slope)

#if ask which kind of customers
Customer_Performance(0,da)

In [ ]:
temp = df.loc[:,['Customers ID','Time']]
visualizable_feature_names = temp.columns #

labels = df['Emotion'].unique() 

import plotly.graph_objects as go

fig_hist = []
for i, feature_name in enumerate(visualizable_feature_names):
    fig_hist.append(go.Figure())
    for label in labels:
        fig_hist[i].add_trace(go.Histogram(x=df[df["Emotion"]==label][feature_name], name=label))
    fig_hist[i].update_layout(height=400, width=800, title_text=feature_name)
    fig_hist[i].update_layout(barmode='overlay')
    fig_hist[i].update_traces(opacity=0.5)
    fig_hist[i].show()